In [ ]:
import subprocess
import sys

def install_packages():
    packages = [
        'streamlit',
        'pandas',
        'numpy',
        'plotly',
        'textblob',
        'vaderSentiment',
        'wordcloud',
        'matplotlib',
        'seaborn'
    ]
    
    for package in packages:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
            print(f"✅ {package} installed")
        except subprocess.CalledProcessError:
            print(f"❌ Failed to install {package}")

install_packages()

In [ ]:
import tweepy
import pandas as pd
from datetime import datetime
import os
import json
import time
from typing import List, Dict

class TwitterClient:
    def __init__(self):
        self.bearer_token = os.getenv('TWITTER_BEARER_TOKEN')
        self.api_key = os.getenv('TWITTER_API_KEY')
        self.api_secret = os.getenv('TWITTER_API_SECRET')
        self.access_token = os.getenv('TWITTER_ACCESS_TOKEN')
        self.access_token_secret = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
        
        if not self.bearer_token:
            print("Twitter API credentials not found. Using simulated data.")
            self.client = None
        else:
            try:
                self.client = tweepy.Client(
                    bearer_token = self.bearer_token,
                    consumer_key = self.api_key,
                    consumer_secret = self.api_secret,
                    access_token = self.access_token,
                    access_token_secret = self.access_token_secret,
                    wait_on_rate_limit=True
                )
                print("✅ Twitter API client initialized successfully")
            except Exception as e:
                print(f"❌ Error initialized Twitter client: {e}")
                self.client = None
                
    def search_tweets(self, query: str, max_results: int = 10) -> List[Dict]:
        if not self.client:
            return self._get_simulated_tweets(max_results)
        
        try:
            tweets = self.client.search_recent_tweets(
                query=query,
                max_results=max_results,
                tweet_fields=['created_at', 'author_id', 'public_metrics', 'context_annotations']  
            )
            
            if not tweets.data:
                return []
            
            tweet_list = []
            for tweet in tweets.data:
                tweet_data = {
                    'text': tweet.text,
                    'timestamp': tweet.created_at,
                    'user': f"user_{tweet.author_id}",
                    'likes': tweet.public_metrics['like_count'] if tweet.public_metrics else 0,
                    'retweets': tweet.public_metrics['retweet_count'] if tweet.public_metrics else 0,
                    'tweet_id': tweet.id
                }
                tweet_list.append(tweet_data)
                
            return tweet_list
        
        except Exception as e:
            print(f"Error fetching tweets: {e}")
            return self._get_simulated_tweets(max_results)
        
    def _get_simulated_tweets(self, count: int) -> List[Dict]:
        from datetime import datetime, timedelta
        import random
        
        sample_tweets = [
            "Just discovered this amazing new coffee shop! ☕ #coffee #love",
            "Traffic is terrible today! 😤 #frustrated #commute",
            "Beautiful sunset tonight 🌅 #nature #peaceful",
            "My flight got delayed again... 😞 #travel #delays",
            "Excited for the weekend! 🎉 #happy #weekend",
            "This movie is absolutely incredible! Must watch 🍿 #movies",
            "Worst customer service ever! Very disappointed 😠 #complaint",
            "Learning Python is so rewarding! 💻 #coding #tech",
            "Rain ruined my picnic plans ☔ #weather #sad",
            "Just finished a great workout! 💪 #fitness #health"
        ]
        
        tweets = []
        for _ in range(count):
            tweet = {
                'text': random.choice(sample_tweets),
                'timestamp': datetime.now() - timedelta(seconds=random.randint(0, 3600)),
                'user': f"user_{random.randint(1000, 9999)}",
                'likes': random.randint(0, 100),
                'retweets': random.randint(0, 50),
                'tweet_id': f"sim_{random.randint(100000, 999999)}"
            }
            tweets.append(tweet)
        
        return tweets


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
from collections import Counter
from wordcloud import WordCloud
import asyncio
import threading

print("✅ Libraries Imported Successfully")

In [ ]:
class EnhancedSentimentAnalyzer:
    def __init__(self) -> None:
        from textblob import TextBlob
        from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
        import re
        
        self.vader_analyzer = SentimentIntensityAnalyzer()
        
    def clean_text(self, text):
        import re
        
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'@\w+', '', text)
        text = re.sub(r'#\w+', '', text)
        text = ' '.join(text.split())
        
        return text.strip()
        
    def analyze_sentiment(self, text):
        from textblob import TextBlob
        
        clean_text = self.clean_text(text)
        
        blob = TextBlob(clean_text)
        textblob_polarity = blob.sentiment.polarity
        
        vader_scores = self.vader_analyzer.polarity_scores(clean_text)
        vader_compound = vader_scores['compound']
        
        if textblob_polarity > 0.1 and vader_compound > 0.05:
            sentiment = 'Positive'
            confidence = (abs(textblob_polarity) + abs(vader_compound)) / 2
        elif textblob_polarity < -0.1 and vader_compound < -0.05:
            sentiment = 'Negative'
            confidence = (abs(textblob_polarity) + abs(vader_compound)) / 2
        else:
            sentiment = 'Neutral'
            confidence = 1 - abs(textblob_polarity - vader_compound)
            
        return {
            'sentiment': sentiment,
            'confidence': min(confidence, 1.0),
            'textblob_polarity': textblob_polarity,
            'vader_compound': vader_compound
        }
        
def main():
    st.set_page_config(
        page_title="Real-Time Twitter Sentiment Analysis",
        page_icon="🐦",
        layout="wide",
        initial_sidebar_state="expanded"
    )
    
    # Custom CSS
    st.markdown("""
    <style>
    .metric-card {
        background-color: #f0f2f6;
        padding: 1rem;
        border-radius: 0.5rem;
        margin: 0.5rem 0;
    }
    .positive-sentiment {
        color: #28a745;
        font-weight: bold;
    }
    .negative-sentiment {
        color: #dc3545;
        font-weight: bold;
    }
    .neutral-sentiment {
        color: #6c757d;
        font-weight: bold;
    }
    </style>
    """, unsafe_allow_html=True)
    
    st.title("🥹 Real-Time Twitter Sentiment Analysis Dashboard")
    st.markdown("---")
    
    if 'twitter_client' not in st.session_state:
        st.session_state.twitter_client = TwitterClient()
    if 'sentiment_analyzer' not in st.session_state:
        st.session_state.sentiment_analyzer = EnhancedSentimentAnalyzer()
    if 'tweets_data' not in st.session_state:
        st.session_state.tweets_data = []
    if 'search_query' not in st.session_state:
        st.session_state.search_query = "python OR javascript OR coding"
        
    st.sidebar.header("🔧 Controls")
    
    # Search configuration
    st.sidebar.subheader("Search Configuration")
    search_query = st.sidebar.text_input(
        "Search Query:", 
        value=st.session_state.search_query,
        help="Enter keywords to search for tweets. Use OR, AND for complex queries."
    )
    
    if search_query != st.session_state.search_query:
        st.session_state.search_query = search_query
        
    tweet_count = st.sidebar.slider("Tweets per fetch:", 10, 100, 20)
    
    auto_refresh = st.sidebar.checkbox("🔄 Auto Refresh", value=False)
    if auto_refresh:
        refresh_interval = st.sidebar.slider("Refresh Interval (seconds):", 30, 300, 60)
    
    if st.sidebar.button("🔍 Fetch New Tweets", type="primary"):
        fetch_tweets(search_query, tweet_count)
        
    if st.sidebar.button("🗑️ Clear All Data"):
        st.session_state.tweets_data = []
        st.rerun()
        
    st.sidebar.subheader("🗃️ Analyze Your Tweet")
    user_tweet = st.sidebar.text_area("Enter your text:")
    if st.sidebar.button("Analyze Text") and user_tweet:
        analyze_user_tweet(user_tweet)
        
    if not st.session_state.tweets_data:
        st.info("👆 Click 'Fetch New Tweets' to start analyzing Twitter sentiment!")
        fetch_tweets(search_query, tweet_count)
    else:
        display_dashboard()

    if auto_refresh and st.session_state.tweets_data:
        placeholder = st.empty()
        with placeholder.container():
            st.info(f"🔄️ Auto-refreshing every {refresh_interval} seconds...")
            time.sleep(refresh_interval)
            fetch_tweets(search_query, tweet_count)
            st.rerun()
            
def fetch_tweets(query, count):
    with st.spinner("🔍 Fetching tweets..."):
        try:
            tweets = st.session_state.twitter_client.search_tweets(query, count)
             
            if not tweets:
                st.warning("No tweets found for the given query.")
                return 
             
            for tweet in tweets:
                analysis = st.session_state.sentiment_analyzer.analyze_sentiment(tweet['text'])
                 
                tweet_data = {
                     'text': tweet['text'],
                     'timestamp': tweet['timestamp'],
                     'user': tweet['user'],
                     'likes': tweet['likes'],
                     'retweets': tweet['retweets'],
                     'sentiment': analysis['sentiment'],
                     'confidence': analysis['confidence'],
                     'textblob_polarity': analysis['textblob_polarity'],
                     'vader_compound': analysis['vader_compound'],
                     'tweet_id': tweet.get('tweet_id', 'unknown')
                }
                 
                st.session_state.tweets_data.append(tweet_data)
                 
            if len(st.session_state.tweets_data) > 500:
                st.session_state.tweets_data = st.session_state.tweets_data[-500:]
            
            st.success(f"✅ Fetched and analyzed {len(tweets)} tweets!")
        
        except Exception as e:
            st.error(f"❌ Error fetching tweets: {str(e)}")
            
def analyze_user_tweet(text):
    analysis = st.session_state.sentiment_analyzer.analyze_sentiment(text)
    
    sentiment_color ={
        'Positive': '🟢',
        'Negative': '🔴', 
        'Neutral': '🟡'
    }    
    
    st.sidebar.success(f"{sentiment_color[analysis['sentiment']]} Sentiment: **{analysis['sentiment']}**")
    st.sidebar.info(f" Confidence: **{analysis['confidence']:.2f}**")
    
    tweet_data = {
        'text': text,
        'timestamp': datetime.now(),
        'user': 'You',
        'likes': 0,
        'retweets': 0,
        'sentiment': analysis['sentiment'],
        'confidence': analysis['confidence'],
        'textblob_polarity': analysis['textblob_polarity'],
        'vader_compound': analysis['vader_compound'],
        'tweet_id': 'user_input'
    }
    
    st.session_state.tweets_data.append(tweet_data)
    
def display_dashboard():
    df = pd.DataFrame(st.session_state.tweets_data)
    
    display_metrics(df)
    
    display_charts(df)
    
    display_recent_tweets(df)
    
def display_metrics(df):
    col1, col2, col3, col4 = st.columns(4)
    
    total_tweets = len(df)
    sentiment_counts = df['sentiment'].value_counts()
    
    with col1:
        st.metric("📊 Total Tweets", total_tweets)
    
    with col2:
        positive_pct = (sentiment_counts.get('Positive', 0) / total_tweets) * 100
        st.metric("😊 Positive", f"{positive_pct:.1f}%", 
                 delta=f"{sentiment_counts.get('Positive', 0)} tweets")
    
    with col3:
        negative_pct = (sentiment_counts.get('Negative', 0) / total_tweets) * 100
        st.metric("😞 Negative", f"{negative_pct:.1f}%",
                 delta=f"{sentiment_counts.get('Negative', 0)} tweets")
    
    with col4:
        avg_confidence = df['confidence'].mean()
        st.metric("🎯 Avg Confidence", f"{avg_confidence:.2f}")
        
def display_charts(df):
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("📈 Sentiment Distribution")
        sentiment_counts = df['sentiment'].value_counts()
        
        colors = {'Positive': '#28a745', 'Negative': '#dc3545', 'Neutral': '#6c757d'}
        fig = px.pie(
            values = sentiment_counts.values,
            names = sentiment_counts.index,
            color = sentiment_counts.index,
            color_discrete_map = colors,
            title = "Sentiment Breakdown"
        )
        fig.update_traces(textposition='inside', textinfo='percent+label')
        st.plotly_chart(fig, use_container_width=True)
        
    with col2:
        st.subheader("⌛ Sentiment Timeline")
        df_sorted = df.sort_values('timestamp').tail(50)
        
        fig = px.scatter(
            df_sorted,
            x='timestamp',
            y='confidence',
            color='sentiment',
            color_discrete_map=colors,
            title='Sentiment Confidence Over Time',
            hover_data=['text']
        )
        fig.update_layout(xaxis_title="Time", yaxis_title="Confidence Score")
        st.plotly_chart(fig, use_container_width=True)
        
    st.subheader("💭 Word Cloud")
    create_wordcloud(df)
    
    st.subheader("📊 Confidence Score Distribution")
    fig = px.violin(
        df,
        x = 'sentiment',
        y = 'confidence',
        color = 'sentiment',
        color_discrete_map = colors,
        title = 'Confidence Score Distribution by Sentiment'
    )
    st.plotly_chart(fig, use_container_width=True)
    
def create_wordcloud(df):
    """Create and display word cloud"""
    try:
        all_text = ' '.join(df['text'].tolist())
        
        if all_text.strip():
            wordcloud = WordCloud(
                width=800,
                height=400,
                background_color='white',
                colormap='viridis',
                max_words=100,
                relative_scaling=0.5
            ).generate(all_text)
            
            fig, ax = plt.subplots(figsize=(10, 5))
            ax.imshow(wordcloud, interpolation='bilinear')
            ax.axis('off')
            st.pyplot(fig)
        else:
            st.info("Not enough text data for word cloud")
    except Exception as e:
        st.error(f"Error creating word cloud: {e}")

def display_recent_tweets(df):
    """Display recent tweets in a table"""
    st.subheader("🐦 Recent Tweets")
    
    df_display = df.sort_values('timestamp', ascending=False).head(20).copy()
    
    df_display['Time'] = pd.to_datetime(df_display['timestamp']).dt.strftime('%H:%M:%S')
    df_display['Tweet'] = df_display['text'].str[:100] + '...'
    df_display['Engagement'] = df_display['likes'] + df_display['retweets']
    
    display_cols = ['Time', 'Tweet', 'sentiment', 'confidence', 'Engagement']
    df_show = df_display[display_cols].copy()
    df_show['confidence'] = df_show['confidence'].round(3)
    
    def style_sentiment(val):
        if val == 'Positive':
            return 'background-color: #d4edda; color: #155724'
        elif val == 'Negative':
            return 'background-color: #f8d7da; color: #721c24'
        else:
            return 'background-color: #e2e3e5; color: #383d41'
        
    styled_df = df_show.style.applymap(style_sentiment, subset=['sentiment'])
    st.dataframe(styled_df, use_container_width=True, height=400)
    
if __name__ == "__main__":
    main()      

In [ ]:
def run_dashboard():
    """Run the Streamlit dashboard"""
    print("🚀 Starting Real-Time Sentiment Analysis Dashboard...")
    print("📊 The dashboard will open in your browser")
    print("---")
    
    main()

run_dashboard()